In [2]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog,sift
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local SIFT 
#Fonction pour extraire les fonctionnalités de SIFT ainsi que les fonctions de HOG à partir d'une image
#def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
#    img = io.imread(file_path)
#    img_gray = rgb2gray(img)
#    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
#    descs = sift(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
#    descs_num = descs.shape[0] * descs.shape[1]
#    sift_desriptors=descs.reshape(descs_num,descs.shape[2])
#  
#    return sift_desriptors,hog_desriptor

#Fonction pour extraire les fonctionnalités de DAISY
def extract_daisy_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
  
    return daisy_desriptors


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend plusieurs minutes pour fonctionner sur un ordinateur  standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features=extract_daisy_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
classifier=extract_daisy_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
pred=classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, pred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,pred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

ImportError: cannot import name 'sift'

In [4]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog#,sift
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local SIFT 
#Fonction pour extraire les fonctionnalités de SIFT ainsi que les fonctions de HOG à partir d'une image
#def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
#    img = io.imread(file_path)
#    img_gray = rgb2gray(img)
#    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
#    descs = sift(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
#    descs_num = descs.shape[0] * descs.shape[1]
#    sift_desriptors=descs.reshape(descs_num,descs.shape[2])
#  
#    return sift_desriptors,hog_desriptor

#Fonction pour extraire les fonctionnalités de DAISY
def extract_daisy_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
  
    return daisy_desriptors


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend plusieurs minutes pour fonctionner sur un ordinateur  standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features=extract_daisy_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
classifier=extract_daisy_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
pred=classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, pred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,pred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier de kmean...
...Chargement du fichier apprentissage...
true label: 10.0


UnboundLocalError: local variable 'hog_feature' referenced before assignment

In [6]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog#,sift
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local SIFT 
#Fonction pour extraire les fonctionnalités de SIFT ainsi que les fonctions de HOG à partir d'une image
#def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
#    img = io.imread(file_path)
#    img_gray = rgb2gray(img)
#    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
#    descs = sift(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
#    descs_num = descs.shape[0] * descs.shape[1]
#    sift_desriptors=descs.reshape(descs_num,descs.shape[2])
#  
#    return sift_desriptors,hog_desriptor

#Fonction pour extraire les fonctionnalités de DAISY
def extract_daisy_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
  
    return daisy_desriptors


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend plusieurs minutes pour fonctionner sur un ordinateur  standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features=extract_daisy_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    #hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)#+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
classifier=extract_daisy_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
pred=classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, pred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,pred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier de kmean...
...Chargement du fichier apprentissage...
true label: 6.0


AttributeError: 'list' object has no attribute 'predict'

In [8]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog#,sift
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local SIFT 
#Fonction pour extraire les fonctionnalités de SIFT ainsi que les fonctions de HOG à partir d'une image
#def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
#    img = io.imread(file_path)
#    img_gray = rgb2gray(img)
#    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
#    descs = sift(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
#    descs_num = descs.shape[0] * descs.shape[1]
#    sift_desriptors=descs.reshape(descs_num,descs.shape[2])
#  
#    return sift_desriptors,hog_desriptor

#Fonction pour extraire les fonctionnalités de DAISY
def extract_daisy_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
  
    return daisy_desriptors


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend plusieurs minutes pour fonctionner sur un ordinateur  standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features=extract_daisy_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    #hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)#+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_hog_hybrid_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
pred=classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, pred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,pred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier de kmean...
...Chargement du fichier apprentissage...
true label: 8.0


ValueError: X.shape[1] = 600 should be equal to 2760, the number of features at training time

In [10]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog#,sift
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local SIFT 
#Fonction pour extraire les fonctionnalités de SIFT ainsi que les fonctions de HOG à partir d'une image
#def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
#    img = io.imread(file_path)
#    img_gray = rgb2gray(img)
#    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
#    descs = sift(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
#    descs_num = descs.shape[0] * descs.shape[1]
#    sift_desriptors=descs.reshape(descs_num,descs.shape[2])
#  
#    return sift_desriptors,hog_desriptor

#Fonction pour extraire les fonctionnalités de DAISY
def extract_daisy_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
  
    return daisy_desriptors


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend plusieurs minutes pour fonctionner sur un ordinateur  standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features=extract_daisy_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    #hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)#+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
hybrid_feature_vector=extract_daisy_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',classifier.predict([hybrid_feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
pred=classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,hybridpred))
print (classification_report(YTEST, pred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,pred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier de kmean...
...Chargement du fichier apprentissage...
true label: 12.0


ValueError: X.shape[1] = 600 should be equal to 2760, the number of features at training time

In [12]:
# %load scenereclassificationTest.py

#%load scenereclassificationTest.py
from os.path import isfile, join, exists
import os.path
from zipfile import ZipFile
from urllib.request import urlretrieve
from os import listdir
from sklearn.externals import joblib
#Pour la division de l'ensemble de données dans un ensemble de formation (100 instances) et un ensemble de test (le reste) #
from sklearn.model_selection import train_test_split
# Routines de traitement d'image pour extraction / transformation de fonctionnalités
from skimage.feature import daisy,hog#,sift
from skimage import io
from skimage.color import rgb2gray
import skimage
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

#Télécharger le jeu de données
if not exists('SceneClass13/'):
    if not exists('SceneClass13.rar'):
        print ('Téléchargement de SceneClass13.rar')
        urlretrieve ('http://vision.stanford.edu/Datasets/SceneClass13.rar')
        print ('Téléchargement de SceneClass13.rar')
    print ('Extraction de SceneClass13.rar')
    zipfile = ZipFile('SceneClass13.rar', 'r')
    zipfile.extractall('./SceneClass13')
    zipfile.close()
    print ('Déjà extraite SceneClass13.rar')
else:
    print ('Le jeu de données est déjà téléchargé et extrait!')


#Obtenez tous les noms de fichiers (y compris le chemin complet) dans un dossier en tant que liste.
def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if (isfile(join(path, f)) and (f.find("Thumbs.db")==-1))]
    return onlyfiles
# Descripteur local SIFT 
#Fonction pour extraire les fonctionnalités de SIFT ainsi que les fonctions de HOG à partir d'une image
#def extract_daisy_and_hog_features_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
#    img = io.imread(file_path)
#    img_gray = rgb2gray(img)
#    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
#    descs = sift(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
#    descs_num = descs.shape[0] * descs.shape[1]
#    sift_desriptors=descs.reshape(descs_num,descs.shape[2])
#  
#    return sift_desriptors,hog_desriptor

#Fonction pour extraire les fonctionnalités de DAISY
def extract_daisy_from_image(file_path,daisy_step_size=32,daisy_radius=32,hog_pixels_per_cell=16,hog_cells_per_block=1):
    img = io.imread(file_path)
    img_gray = rgb2gray(img)
    img=skimage.transform.resize(img_gray,(300,250)) ##resize to a suitable dimension, avg size of images in the dataset
    #original, histograms=6
    descs = daisy(img, step=daisy_step_size, radius=daisy_radius, rings=2, histograms=8,orientations=8, visualize=False)
    #Calculer les descripteurs de caractéristiques de marguerite
    descs_num = descs.shape[0] * descs.shape[1]
    daisy_desriptors=descs.reshape(descs_num,descs.shape[2])
  
    return daisy_desriptors


base_path="SceneClass13/" #Chemin où les données d'image sont conservées
img_width=300
img_height=250
hog_pixels_per_cell=16
orientations=8
#Chargez les noms de fichiers correspondant à chaque catégorie de scène en listes
category_names=listdir(base_path) ##
for i in range(len(category_names)):
    print (category_names[i],'=',i)
print ('total categories:',len(category_names))
dataset_filenames=[] #Liste pour garder le chemin de tous les fichiers dans la base de données
dataset_labels=[]
##category_names[0] Liste la représentation textuelle de l'identifiant de catégorie
for category in category_names:
    category_filenames=get_filenames(base_path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)
    #for num in range(0,100):
    #    train_filenames = train_filenames + 
print ('Taille totale du jeu de données:',len(dataset_filenames))
#Split into training files and testing files
#sss = StratifiedShuffleSplit(dataset_labels,train_size=100, random_state=0)
#train_index, test_index = [s for s in sss.split(dataset_filenames,dataset_labels)][0]
#train_filenames, test_filenames, train_labels, test_labels = dataset_filenames[train_index, :], dataset_labels[train_index], dataset_filenames[test_index, :], dataset_labels[test_index]
#for train_index, test_index in sss:
#    print("TRAIN:", train_index, "TEST:", test_index)
#    train_filenames, test_filenames = dataset_filenames[train_index], dataset_filenames[test_index]
#    train_labels, test_labels = dataset_labels[train_index], dataset_labels[test_index]
print ('Total des fichiers du jeu de données :',len(dataset_filenames))
print ('Total des labels dans le jeu de données:',len(dataset_labels))
train_filenames, test_filenames, train_labels, test_labels = train_test_split(dataset_filenames,dataset_labels,train_size=1300, stratify=dataset_labels)
#train_filenames,test_filenames,train_labels,test_labels=train_test_split(dataset_filenames,dataset_labels,train_size =100,stratify=dataset_labels)
print ('Nombre total des fichiers du split entrainement:',len(train_filenames))
print ('Nombre total des fichiers du split de test:',len(test_filenames))
#Extraire les fonctionnalités de la division de données de formation pour le traitement en aval, prend environ 12 minutes pour un ordinateur portable standard
training_data_feature_map={} #map pour stocker les caractéristiques DAISY ainsi que la caractéristique HOG pour tous les points de données de entrainement
daisy_descriptor_list=[] #Liste pour stocker tous les descripteurs DAISY pour former notre vocabulaire visuel en regroupant
counter=0

#Maintenant, pour former des "mots visuels", nous agrandissons les caractéristiques de DAISY pour former un vocabulaire, nous formons une caractéristique d'histogramme (histogramme de DAISY) correspondant à chaque caractéristique des dimensions 'number_of_clusters'

#Entrée: liste des caractéristiques DAISY  et nombre de clusters
#Sorite: a trained cluster model which will allow to get the cluster id corresponding to any input daisy feature
def cluster_daisy_features(daisy_feature_list,number_of_clusters):
    #km=KMeans(n_clusters=number_of_clusters)
    km=MiniBatchKMeans(n_clusters=number_of_clusters,batch_size=number_of_clusters*10)
    km.fit(daisy_feature_list)
    return km
# cacher les  warnings
import warnings
warnings.filterwarnings('ignore')
#Le nombre de clusters est défini comme 600 #, prend plusieurs minutes pour fonctionner sur un ordinateur  standard
filename1 = 'kmean.pkl'

if os.path.exists(filename1):
   # load the model from disk
    print ('...Chargement du fichier de kmean...')
    daisy_cluster_model = joblib.load(filename1)
else:
    for fname in tqdm(train_filenames):
        daisy_features,hog_feature=extract_daisy_and_hog_features_from_image(fname,daisy_step_size=8,daisy_radius=8)
    #Extraire les fonctionnalités DAISY et les fonctions HOG de l'image et enregistrer dans une map
        training_data_feature_map[fname]=[daisy_features,hog_feature]
        daisy_descriptor_list=daisy_descriptor_list+list(daisy_features)
    print ('Total daisy descriptors:',len(daisy_descriptor_list))
    print ('...Enregistrement du modèle...')
    daisy_cluster_model=cluster_daisy_features(daisy_descriptor_list,600) 
    daisy_cluster_model.n_clusters
    joblib.dump(daisy_cluster_model, filename1)

#Fonction pour extraire la fonction hybride des images en regroupant l'histogramme de DAISY et le descripteur de HOG après l'individu
def extract_daisy_feature_from_image(fname,daisy_cluster_model):
    #Dans le cas où nous aurions rencontré le fichier lors de l'entrainement, les caractéristiques DAISY et HOG auraient déjà été calculées
    if fname in training_data_feature_map:
        daisy_features=training_data_feature_map[fname][0]
        hog_feature=training_data_feature_map[fname][1]
    else:
        daisy_features=extract_daisy_from_image(fname,daisy_step_size=8,daisy_radius=8)
        
    # Indiquer à quoi appartiennent les cluster de chaque caractéristique DAISY
    img_clusters=daisy_cluster_model.predict(daisy_features) 
    cluster_freq_counts=pd.DataFrame(img_clusters,columns=['cnt'])['cnt'].value_counts()
    bovw_vector=np.zeros(daisy_cluster_model.n_clusters) ##feature vector of size as the total number of clusters
    for key in cluster_freq_counts.keys():
        bovw_vector[key]=cluster_freq_counts[key]

    bovw_feature=bovw_vector/np.linalg.norm(bovw_vector)
    #hog_feature=hog_feature/np.linalg.norm(hog_feature)
    return list(bovw_feature)#+list(hog_feature)

def plot_file(fname):
    img_data=plt.imread(fname)
    plt.imshow(rgb2gray(img_data),cmap='Greys_r')
#Extraction de caractéristiques de données d'entrainement
XTRAIN=[]
YTRAIN=[]
filename = 'trainedsvm.pkl'

if os.path.exists(filename):
   # load the model from disk
    print ('...Chargement du fichier apprentissage...')
    classifier = joblib.load(filename)
else:
    for i in tqdm(range(len(train_filenames))):
        XTRAIN.append(extract_daisy_feature_from_image(train_filenames[i],daisy_cluster_model))
        YTRAIN.append(train_labels[i])
    classifier=svm.SVC(C=10**1.6794140624999994, gamma=10**-0.1630955304365928, decision_function_shape='ovo') #paramètres de la fonction noyau SVM
    print ('...Enregistrement du modèle...')
    classifier.fit(XTRAIN,YTRAIN)
    print ('...Enregistrement du fichier apprentissage...')
    joblib.dump(classifier, filename)

# Montrer un example de classification
plot_file(test_filenames[4])
print ('true label:',test_labels[4])
feature_vector=extract_daisy_feature_from_image(test_filenames[4],daisy_cluster_model)
print ('prediction:',classifier.predict([feature_vector]))

## Test de l'extraction des fonctions de données, ne faites que pour quelques instants si un test rapide est nécessaire
XTEST=[]
YTEST=[]
for i in tqdm(range(len(test_filenames))):
    XTEST.append(extract_daisy_feature_from_image(test_filenames[i],daisy_cluster_model))
    YTEST.append(test_labels[i])
print ('Métriques du classifieur hybride')
print ('Nombre instances test:',len(XTEST),len(YTEST))
#Rapport de précision
pred=classifier.predict(XTEST)
       
print ('Précision globale: \n',accuracy_score(YTEST,pred))
print (classification_report(YTEST, pred, target_names=category_names))
print ('Matrice de confusion:\n')
pd.DataFrame(confusion_matrix(YTEST,pred),
            columns=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb'],
               index=['MITinsidecity', 'bedroom', 'PARoffice', 'MITmountain', 'MITtallbuilding', 'MIThighway', 'MITcoast', 'livingroom', 'MITopencountry', 'MITstreet', 'MITforest', 'kitchen', 'CALsuburb']) 

Le jeu de données est déjà téléchargé et extrait!
bedroom = 0
CALsuburb = 1
kitchen = 2
livingroom = 3
MITcoast = 4
MITforest = 5
MIThighway = 6
MITinsidecity = 7
MITmountain = 8
MITopencountry = 9
MITstreet = 10
MITtallbuilding = 11
PARoffice = 12
total categories: 13
Taille totale du jeu de données: 3859
Total des fichiers du jeu de données : 3859
Total des labels dans le jeu de données: 3859
Nombre total des fichiers du split entrainement: 1300
Nombre total des fichiers du split de test: 2559
...Chargement du fichier de kmean...


100%|██████████████████████████████████████| 1300/1300 [06:54<00:00,  2.96it/s]


...Enregistrement du modèle...
...Enregistrement du fichier apprentissage...
true label: 9.0
prediction: [ 9.]


100%|██████████████████████████████████████| 2559/2559 [12:35<00:00,  3.08it/s]


Métriques du classifieur hybride
Nombre instances test: 2559 2559
Précision globale: 
 0.752637749121
                 precision    recall  f1-score   support

        bedroom       0.49      0.43      0.46       143
      CALsuburb       0.94      0.94      0.94       160
        kitchen       0.52      0.52      0.52       139
     livingroom       0.49      0.54      0.51       192
       MITcoast       0.76      0.86      0.81       239
      MITforest       0.84      0.91      0.87       217
     MIThighway       0.83      0.81      0.82       172
  MITinsidecity       0.77      0.73      0.75       204
    MITmountain       0.84      0.81      0.83       248
 MITopencountry       0.73      0.72      0.73       272
      MITstreet       0.83      0.79      0.81       194
MITtallbuilding       0.87      0.82      0.84       236
      PARoffice       0.74      0.71      0.73       143

    avg / total       0.75      0.75      0.75      2559

Matrice de confusion:



MITinsidecity  bedroom  PARoffice  MITmountain  \
MITinsidecity               62        1         16           38   
bedroom                      1      150          0            4   
PARoffice                   21        0         72           30   
MITmountain                 31        1         29          104   
MITtallbuilding              0        0          0            0   
MIThighway                   0        1          0            0   
MITcoast                     0        1          0            1   
livingroom                   0        0          8            1   
MITopencountry               1        0          0            1   
MITstreet                    0        1          0            0   
MITforest                    0        5          1            2   
kitchen                      6        0          0           10   
CALsuburb                    5        0         12           23   

                 MITtallbuilding  MIThighway  MITcoast  livingroom  \
MITinsidecity                  1           0         0           3   
bedroom                        0           0         0           1   
PARoffice                      0           0         0           7   
MITmountain                    1           0         0           6   
MITtallbuilding              205           0         8           0   
MIThighway                     1         198         0           0   
MITcoast                      10           2       139           5   
livingroom                     4           3         5         148   
MITopencountry                 9          10         3           0   
MITstreet                     33          16         6           1   
MITforest                      1           1         6          12   
kitchen                        3           6         0          10   
CALsuburb                      0           0         0           0   

                 MITopencountry  MITstreet  MITforest  kitchen  CALsuburb  
MITinsidecity                 2          1          2        4         13  
bedroom                       0          1          2        1          0  
PARoffice                     0          0          1        0          8  
MITmountain                   1          1          0        4         14  
MITtallbuilding               1         25          0        0          0  
MIThighway                    8          7          2        0          0  
MITcoast                      2          9          3        0          0  
livingroom                    0          5         16       13          1  
MITopencountry              202         19          3        0          0  
MITstreet                    16        197          1        1          0  
MITforest                     5          2        153        6          0  
kitchen                       4          2          1      194          0  
CALsuburb                     0          0          0        1        102